In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.storage import LocalFileStore
from langchain.schema import Document
import os

In [2]:
llm = ChatOpenAI(temperature=0.1, streaming=True)

# LLM을 활용한 문서 요약 함수
def summarize_documents(docs, llm):
    summarized_docs = []
    for doc in docs:
        # 요약 요청을 LLM으로 보내기
        summary = llm.predict(f"이 내용 요약해줘: {doc.page_content}")
        
        # Document 객체에 담아 summarized_docs에 추가 
        summarized_docs.append(Document(page_content=summary))
    return summarized_docs

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22464\3616085251.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1, streaming=True)


In [ ]:
def embed_file(file):
    
    file_content = file.read()  # 파일의 내용을 읽어오기
    file_path = f"./.cache/files/{file.name}"  # 저장될 파일의 경로
    os.makedirs(file_path, exist_ok=True) # 디렉토리가 없으면 만들기
    with open(file_path, "wb") as f:
        f.write(file_content)  # 선택한 파일의 내용을 .cache/files 디렉토리로 옮김

    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
    )

    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".[:10] | .[].content",  # 10개의 기사만 가져오기
        text_content=True,
    )

    data = loader.load()
    
    docs = splitter.split_documents(data)  # splitter에 맞게 문서 분할
    
    # 1.요약하기 버튼을 선택한 경우
    # 요약한 문서의 임베딩 결과를 저장할 디렉토리 만들기
    summary_cache_dir_path = f"./.cache/summarized_embeddings/{file.name}"
    os.makedirs(summary_cache_dir_path, exist_ok=True) # 디렉토리가 없으면 만들기
    summary_cache_dir = LocalFileStore(summary_cache_dir_path)
    
    # 문서 요약
    summary_docs = summarize_documents(docs, llm)
    summary_docs_path = f"./.cache/summarized_docs/{file.name}"
    os.makedirs(summary_docs_path, exist_ok=True) # 디렉토리가 없으면 만들기
    with open(summary_docs_path, "wb") as f:
        f.write(summary_docs)
    
    summary_embeddings = OpenAIEmbeddings()

    # 중복 요청 시 캐시된 결과를 반환
    summary_cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        summary_embeddings, summary_cache_dir
    )
    
    # 2.요약하기 버튼을 선택하지 않은 경우
    # 문서의 임베딩 결과를 저장할 디렉토리 만들기
    cache_dir_path = f"./.cache/embeddings/{file.name}"
    os.makedirs(cache_dir_path, exist_ok=True) # 디렉토리가 없으면 만들기
    cache_dir = LocalFileStore(cache_dir_path)

    embeddings = OpenAIEmbeddings()

    # 중복 요청 시 캐시된 결과를 반환
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        embeddings, cache_dir
    )

SyntaxError: expected ':' (924623980.py, line 35)